In [ ]:
from pyspark.sql import SparkSession

Superstore pre-processing

In [ ]:
spark = SparkSession.builder \
    .appName("DeltaLake") \
    .master("local[*]") \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0,io.delta:delta-spark_2.13:4.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

df_superstore = spark.read.format("delta").load("hdfs://localhost:9000/delta_superstore")
print(df_superstore)

In [ ]:
df_superstore.show()